In [ ]:
# 구글 번역 api (처음 한번만 설치)
%pip install googletrans==4.0.0-rc1

In [39]:
# 이 파일 있는 디렉터리로 이동??
import os
os.chdir(os.getcwd())
%pwd

#%cd /content/drive/MyDrive/Github/floread/sentiment-analysis

'c:\\Users\\KangIW\\git_j500\\floread'

In [9]:
# 가장 퀄리티가 나은 감성대화말뭉치 사용
import pandas as pd

df = pd.read_csv('data/감성대화말뭉치(병합).csv',index_col=0)
df.shape

(58240, 2)

In [11]:
df['emotion'] = df['emotion'].replace('상처', '슬픔')
df.groupby('emotion').count().iloc[:,0]

emotion
기쁨     7337
당황     9797
분노    10414
불안    10429
슬픔    20263
Name: sentence, dtype: int64

In [8]:
import re

#한글 있는지 체크
def has_korean(text):
    korean_check = lambda text: any(char >= '가' and char <= '힣' for char in text)
    return korean_check(text)

#0으로 끝나는 문장
def remove_end_0(sentence):
    if sentence.endswith('0'):
        return sentence[:-1]
    return sentence

def Preprocess_all(df):
    
    print("---------------전처리 전: ",df.shape)
    
    # 결측치 제거
    df = df.dropna()
    
    #필터 처리된 단어 공백으로 치환
    df['sentence'] = df['sentence'].str.replace(r'OO+', ' ', regex=True)
    
    #좌측 공백 제거
    df['sentence'] = df['sentence'].str.lstrip()
    
    #한글 없는 문장 제거
    df = df[df['sentence'].apply(lambda x: has_korean(x))]
    
    #한자 들어간 문장 제거
    df = df[~df['sentence'].str.contains('[\u4e00-\u9fff]')]
        
    #일부 단어 치환
    df['sentence'] = df['sentence'].str.replace(r'(ㅋ{3,})', 'ㅋㅋ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'\?\?{3,}', '??,', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'~{3,}', '~~', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'!{3,}', '!!', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'-{3,}', '--', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㅠ{3,})', 'ㅠㅠ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㅜ{3,})', 'ㅜㅜ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㄱ{3,})', 'ㄱㄱ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㄷ{3,})', 'ㄷㄷ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㅎ{3,})', 'ㅎㅎ', regex=True)

    df['sentence'] = df['sentence'].str.replace(r'(♥{2,})', '♥', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(♡{2,})', '♡', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'\s{2,}', ' ', regex=True)
    
    df['sentence'] = df['sentence'].str.replace(r'\.{7,}', '......', regex=True)
    df['sentence'] = df['sentence'].str.replace(';{3,}', ';;')
    df['sentence'] = df['sentence'].str.replace('\^{3,}', '^^')
    
    # 한글 자모음이 3번 이상 반복되는 패턴을 찾아, 그 자모음 2개로 치환
    df['sentence'] = df['sentence'].apply(lambda x: re.sub(r'(([ㄱ-ㅎㅏ-ㅣ]|ㄲ|ㄸ|ㅃ|ㅆ|ㅉ){3,})', lambda m: m.group(1)[:2], x))

    # 길이가 100이상인 문장 제거
    df = df[df['sentence'].str.len() < 100]
    
    # 0으로 끝나면 0빼기
    df['sentence'] = df['sentence'].apply(remove_end_0)
    
    #의미 없는 0패턴 제거
    replacements = {
        r'\?0': '?',
        r';0': ';',
        r'ㅠ0': 'ㅠ',
        r'ㅜ0': 'ㅜ',
        r'\.0': '.'
    }

    for pattern, replacement in replacements.items():
        df['sentence'] = df['sentence'].str.replace(pattern, replacement, regex=True)
        
    # 중복행 제거
    dup = df[df.duplicated(subset=['sentence'])]
    print("중복되는 행: ", len(dup))
    df = df.drop_duplicates(subset=['sentence'], keep='first')
    
    # 인덱스 재설정
    df.reset_index(drop=True, inplace=True)

    print("---------------전처리 후: ",df.shape)    
    
    return df

In [16]:
pp_df = Preprocess_all(df)

---------------전처리 전:  (58240, 2)
중복되는 행:  0
---------------전처리 후:  (58224, 2)


In [ ]:
pp_df.to_csv('data/감성대화말뭉치_전처리완.csv', encoding='utf-8-sig')

In [2]:
import pandas as pd
df = pd.read_csv('data/감성대화말뭉치_전처리완.csv',index_col=0)
df.groupby('emotion').count().iloc[:,0]

emotion
기쁨     7334
당황     9794
분노    10410
불안    10428
슬픔    20258
Name: sentence, dtype: int64

In [28]:
dfsum = pd.read_csv('data/병합데이터셋-v3.csv',index_col=0)
print(dfsum.shape)
dfsum.groupby('emotion').count().iloc[:,0]

(135791, 2)


emotion
놀람    19538
분노    19526
불안    15980
슬픔    21305
중립    45149
행복    14293
Name: sentence, dtype: int64

놀람1000,분노1000, 불안4000, 슬픔 -, 중립-, 행복6000

In [3]:
import pandas as pd
df = pd.read_csv('data/감성대화말뭉치_전처리완.csv',index_col=0)
df = df[df['emotion'] != '슬픔']
df.shape

(37966, 2)

In [4]:
from googletrans import Translator
import time

# 전역으로 선언하면 오버해드가 줄어든다?
translator = Translator()

# 역번역 함수: 한글 -> 영어 -> 한글
def backtranslate(text):
    global translator   
    
    en = translator.translate(text, src='ko', dest='en').text
    time.sleep(0.3)   #ReadTimeout 방지
    
    kr = translator.translate(en, src='en', dest='ko').text
    time.sleep(0.3)   
    
    return kr

In [4]:
# 카운트 테스트
from googletrans import Translator
import time

translator = Translator()

cnt=0
# 역번역 함수: 한글 -> 영어 -> 한글
def backtranslate(text):
    global translator   
    global cnt
    
    cnt +=1
    if cnt%100 ==0: print(cnt)
    en = translator.translate(text, src='ko', dest='en').text
    time.sleep(0.5)   #ReadTimeout 방지
    
    kr = translator.translate(en, src='en', dest='ko').text
    time.sleep(0.5)   
    
    return kr

In [5]:
# (에러방지)데이터 프레임 분리 후 각각 번역
df1 = df[:3000]
df2 = df[3000:6000]
df3 = df[6000:9000]
df4 = df[9000:12000]
df5 = df[12000:15000]
df6 = df[15000:18000]
df7 = df[18000:21000]
df8 = df[21000:24000]
df9 = df[24000:27000]
df10 = df[27000:30000]
df11 = df[30000:33000]
df12 = df[33000:36000]
df13 = df[36000:]

In [ ]:
df1['sentence'] = df1['sentence'].apply(backtranslate)
df1.to_csv('res/df1.csv', encoding='utf-8-sig')

In [5]:
df1 = pd.read_csv('data/df1.csv',index_col=0)
df2 = pd.read_csv('data/df2.csv',index_col=0)
df3 = pd.read_csv('data/df3.csv',index_col=0)
df4 = pd.read_csv('data/df4.csv',index_col=0)
df5 = pd.read_csv('data/df5.csv',index_col=0)
df6 = pd.read_csv('data/df6.csv',index_col=0)
df7 = pd.read_csv('data/df7.csv',index_col=0)
df8 = pd.read_csv('data/df8.csv',index_col=0)
df9 = pd.read_csv('data/df9.csv',index_col=0)
df10 = pd.read_csv('data/df10.csv',index_col=0)
df11 = pd.read_csv('data/df11.csv',index_col=0)
df12 = pd.read_csv('data/df12.csv',index_col=0)
df13 = pd.read_csv('data/df13.csv',index_col=0)

In [6]:
import pandas as pd
concat_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13])

In [5]:
concat_df.groupby('emotion').count().iloc[:,0]

emotion
기쁨     7334
당황     9794
분노    10410
불안    10428
Name: sentence, dtype: int64

In [13]:
concat_df.head()

,sentence,emotion
0,일을 끝내지 않는 이유는 무엇입니까? 화난.,분노
1,급여는 이번 달에 다시 삭감되었습니다! 가격이 올라가서 저급을 계속 지불하기 때문에...,분노
2,회사에 신입 사원이 있지만 성가신 일입니다. 매일 그런 아이를 봐야한다고 생각합니다.,분노
3,나는 직장에서 막내 가기 때문에 모든 종류의 심부름을 줘. 많은 일이 있지만 정말 ...,분노
4,최근에 회사에 합류 한 신입 사원이 저를 무시하는 것 같습니다.,분노


In [12]:
import pandas as pd

df = pd.read_csv('data/병합데이터셋-v3.csv',index_col=0)

dfv5 = pd.concat([df, concat_df])
dfv5.sample(n=20)

,sentence,emotion
114728,수영장에 함께 다녀온 아내가 이명이 들린다니 당황스러워.,놀람
69071,다리 모아봐. 떨어뜨리면 안돼.,중립
13897,다행이다... 그럼 잘 설득하면 되는 거네요.,중립
116588,아들이 함께 살기 힘들다고 따로 거처를 마련했다는 말이 절로 가슴 아파.,슬픔
7638,이대호가 뭐가아쉬워서 롯데로 유턴하지??,놀람
51378,수업 시간에 괴롭힘을당한 것 같아서 열등합니다.,분노
17121,내일 어버이연합 틀딱 쓰래기 늙은이들 죠패러갑니다,분노
72529,내일 웨딩사진 촬영 있는 날인데 살이 생각만큼 안 빠져서 완전히 스트레스받아.,불안
17485,친구가 나를 의심해서 너무 속상해.,슬픔
17212,얼마 전 내 팔순을 맞아 경로당에서 잔치를 열어줘서 얼마나 기쁘던지.,행복


In [14]:
Preprocess_all(dfv5)

---------------전처리 전:  (173757, 2)
중복되는 행:  861
---------------전처리 후:  (172885, 2)


,sentence,emotion
0,노년이 되니 자녀 관계가 악화하고 있어.,놀람
1,쓸데없는 소리 말고 가!,분노
2,내 우울증 치료로 돈이 많이 나간다고 아내가 이혼을 요구했어.,슬픔
3,"글쎄, 너랑 같이 안 가면 난 예년대로 혼자 가는 거지 뭐. 그게 홀가분하고 편하니...",중립
4,여기! 도 좀 쓸어!,중립
...,...,...
172880,나와 동시에 결혼 한 친구는 내 집을 부러워하며 대출부터 시작합니다.,당황
172881,친구가 결혼하여 아이가 있었고 너무 행복해 보였습니다. 기분이 좋지 않습니다.,당황
172882,"다른 사람들은 결혼하기 전에 1 억을 모았지만, 내가 한 일을 모르겠습니다. 나는 ...",당황
172883,나는 나보다 먼저 결혼 한 친구를 부러워합니다. 친구의 남편은 남편보다 낫고 키가 ...,당황


In [15]:
dfv5.to_csv('data/병합데이터셋-v5.csv', encoding='utf-8-sig')

In [2]:
import pandas as pd

dfv5 = pd.read_csv('data/병합데이터셋-v5.csv',index_col=0)


In [4]:
dfv5.groupby('emotion').count().iloc[:,0]

emotion
기쁨     7334
놀람    19538
당황     9794
분노    29936
불안    26408
슬픔    21305
중립    45149
행복    14293
Name: sentence, dtype: int64